In [1]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
import keras
import datetime
from tqdm import tqdm
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import BaggingRegressor
from catboost import CatBoostRegressor
import matplotlib.pyplot as plt

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [2]:
#This notebooks is a work in progress and incomplete.

In [3]:
train=pd.read_csv("/kaggle/input/playground-series-s3e19/train.csv",index_col='id')
test=pd.read_csv("/kaggle/input/playground-series-s3e19/test.csv",index_col='id')

In [4]:
train.head()

,date,country,store,product,num_sold
id,,,,,
0,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Improve Your Coding,63
1,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Train More LLMs,66
2,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Win Friends and Influence People,9
3,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Win More Kaggle Competitions,59
4,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Write Better,49


In [5]:
len(train['date'].unique())

1826

In [6]:
cat_features=['country','store','product']
for feature in cat_features:
    print(feature,':',train[feature].unique())

country : ['Argentina' 'Canada' 'Estonia' 'Japan' 'Spain']
store : ['Kaggle Learn' 'Kaggle Store' 'Kagglazon']
product : ['Using LLMs to Improve Your Coding' 'Using LLMs to Train More LLMs'
 'Using LLMs to Win Friends and Influence People'
 'Using LLMs to Win More Kaggle Competitions' 'Using LLMs to Write Better']


In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 136950 entries, 0 to 136949
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   date      136950 non-null  object
 1   country   136950 non-null  object
 2   store     136950 non-null  object
 3   product   136950 non-null  object
 4   num_sold  136950 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 6.3+ MB


In [8]:
def datetime_features(df_temp):
    """
ref:-https://www.kaggle.com/code/chingiznurzhanov/timeseriessplit-catboost-trick
    """
    df_temp['month'] = df_temp['date'].dt.month
    df_temp['day'] = df_temp['date'].dt.day
    df_temp['year'] = df_temp['date'].dt.year
    df_temp['dayofweek'] = df_temp['date'].dt.dayofweek
    df_temp['quarter'] = df_temp['date'].dt.quarter
    df_temp['dayofmonth'] = df_temp['date'].dt.day
    #df_temp['weekofyear'] = df_temp['date'].dt.weekofyear
    df_temp['weekofyear'] = df_temp['date'].dt.isocalendar().week.astype(np.int64)
    df_temp['is_month_start'] = df_temp.date.dt.is_month_start.astype(np.int8)
    df_temp['is_month_end'] = df_temp.date.dt.is_month_end.astype(np.int8)
    df_temp['monday'] = df_temp.date.dt.weekday.eq(0).astype(np.uint8)
    df_temp['tuesday'] = df_temp.date.dt.weekday.eq(1).astype(np.uint8)
    df_temp['wednesday'] = df_temp.date.dt.weekday.eq(2).astype(np.uint8)
    df_temp['thursday'] = df_temp.date.dt.weekday.eq(3).astype(np.uint8)
    df_temp['friday'] = df_temp.date.dt.weekday.eq(4).astype(np.uint8)
    df_temp['saturday'] = df_temp.date.dt.weekday.eq(5).astype(np.uint8)
    df_temp['sunday'] = df_temp.date.dt.weekday.eq(6).astype(np.uint8)
    df_temp['is_quarter_end'] = df_temp['date'].dt.is_quarter_end.astype(np.uint8)
    df_temp['is_quarter_start'] = df_temp['date'].dt.is_quarter_start.astype(np.uint8)
    df_temp['is_year_end'] = df_temp['date'].dt.is_year_end.astype(np.uint8)
    df_temp['is_year_start'] = df_temp['date'].dt.is_year_start.astype(np.uint8)
    df_temp=df_temp.drop('date',axis=1)
    return df_temp

In [9]:
def data_encoding(dataset):
    cat_features=['country','store','product']
    for feature in tqdm(cat_features):
        dictionary={}
        n=1
        print('feature:',feature)
        print('categories:',dataset[feature].unique())
        for sub_cat in dataset[feature].unique():
            dictionary[sub_cat]=n
            n+=1
        print('encoding rule:',dictionary)
        dataset[feature]=dataset[feature].map(dictionary)
        dummies=pd.get_dummies(dataset[feature],prefix=feature)
        for key in dummies:
            dataset[key]=dummies[key]
        dataset=dataset.drop(feature,axis=1)
    dataset['date']=pd.to_datetime(dataset['date'],format='%Y-%m-%d')
    print("Date converted to Datetime features. Check returned dataframe if you want further details.")
    return datetime_features(dataset)

In [10]:
train=data_encoding(train)
test=data_encoding(test)

  0%|          | 0/3 [00:00<?, ?it/s]

feature: country
categories: ['Argentina' 'Canada' 'Estonia' 'Japan' 'Spain']
encoding rule: {'Argentina': 1, 'Canada': 2, 'Estonia': 3, 'Japan': 4, 'Spain': 5}
feature: store
categories: ['Kaggle Learn' 'Kaggle Store' 'Kagglazon']
encoding rule: {'Kaggle Learn': 1, 'Kaggle Store': 2, 'Kagglazon': 3}


100%|██████████| 3/3 [00:00<00:00, 15.07it/s]


feature: product
categories: ['Using LLMs to Improve Your Coding' 'Using LLMs to Train More LLMs'
 'Using LLMs to Win Friends and Influence People'
 'Using LLMs to Win More Kaggle Competitions' 'Using LLMs to Write Better']
encoding rule: {'Using LLMs to Improve Your Coding': 1, 'Using LLMs to Train More LLMs': 2, 'Using LLMs to Win Friends and Influence People': 3, 'Using LLMs to Win More Kaggle Competitions': 4, 'Using LLMs to Write Better': 5}
Date converted to Datetime features. Check returned dataframe if you want further details.


100%|██████████| 3/3 [00:00<00:00, 59.30it/s]

feature: country
categories: ['Argentina' 'Canada' 'Estonia' 'Japan' 'Spain']
encoding rule: {'Argentina': 1, 'Canada': 2, 'Estonia': 3, 'Japan': 4, 'Spain': 5}
feature: store
categories: ['Kaggle Learn' 'Kaggle Store' 'Kagglazon']
encoding rule: {'Kaggle Learn': 1, 'Kaggle Store': 2, 'Kagglazon': 3}
feature: product
categories: ['Using LLMs to Improve Your Coding' 'Using LLMs to Train More LLMs'
 'Using LLMs to Win Friends and Influence People'
 'Using LLMs to Win More Kaggle Competitions' 'Using LLMs to Write Better']
encoding rule: {'Using LLMs to Improve Your Coding': 1, 'Using LLMs to Train More LLMs': 2, 'Using LLMs to Win Friends and Influence People': 3, 'Using LLMs to Win More Kaggle Competitions': 4, 'Using LLMs to Write Better': 5}
Date converted to Datetime features. Check returned dataframe if you want further details.


In [11]:
train.head()

,num_sold,country_1,country_2,country_3,country_4,country_5,store_1,store_2,store_3,product_1,...,tuesday,wednesday,thursday,friday,saturday,sunday,is_quarter_end,is_quarter_start,is_year_end,is_year_start
id,,,,,,,,,,,,,,,,,,,,,
0,63,1,0,0,0,0,1,0,0,1,...,0,0,0,0,0,1,0,1,0,1
1,66,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,1,0,1
2,9,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,1,0,1
3,59,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,1,0,1
4,49,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,1,0,1


In [12]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 136950 entries, 0 to 136949
Data columns (total 34 columns):
 #   Column            Non-Null Count   Dtype
---  ------            --------------   -----
 0   num_sold          136950 non-null  int64
 1   country_1         136950 non-null  uint8
 2   country_2         136950 non-null  uint8
 3   country_3         136950 non-null  uint8
 4   country_4         136950 non-null  uint8
 5   country_5         136950 non-null  uint8
 6   store_1           136950 non-null  uint8
 7   store_2           136950 non-null  uint8
 8   store_3           136950 non-null  uint8
 9   product_1         136950 non-null  uint8
 10  product_2         136950 non-null  uint8
 11  product_3         136950 non-null  uint8
 12  product_4         136950 non-null  uint8
 13  product_5         136950 non-null  uint8
 14  month             136950 non-null  int64
 15  day               136950 non-null  int64
 16  year              136950 non-null  int64
 17  dayofweek 

In [13]:
X,y=train.drop("num_sold",axis=1),train["num_sold"]

In [14]:
#(tf.convert_to_tensor(X),tf.convert_to_tensor(y))

In [15]:
# cb_reg=CatBoostRegressor(loss_function='MAPE',verbose=3)
# cb_reg.fit(X,y)

In [16]:
# bag_reg=BaggingRegressor(cb_reg,n_estimators=5,n_jobs=-1,verbose=3)
# bag_reg.fit(X,y)

In [17]:
#bag_reg.predict(test)

In [18]:
#cb_reg.predict(test)

In [19]:
rf_reg=RandomForestRegressor(n_estimators=1000,n_jobs=-1,verbose=3)
rf_reg.fit(X,y)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


building tree 1 of 1000
building tree 2 of 1000
building tree 3 of 1000
building tree 4 of 1000
building tree 5 of 1000
building tree 6 of 1000
building tree 7 of 1000
building tree 8 of 1000
building tree 9 of 1000
building tree 10 of 1000
building tree 11 of 1000
building tree 12 of 1000
building tree 13 of 1000
building tree 14 of 1000
building tree 15 of 1000
building tree 16 of 1000
building tree 17 of 1000
building tree 18 of 1000
building tree 19 of 1000
building tree 20 of 1000
building tree 21 of 1000
building tree 22 of 1000
building tree 23 of 1000
building tree 24 of 1000
building tree 25 of 1000building tree 26 of 1000
building tree 27 of 1000

building tree 28 of 1000


[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    6.7s


building tree 29 of 1000
building tree 30 of 1000
building tree 31 of 1000
building tree 32 of 1000
building tree 33 of 1000
building tree 34 of 1000
building tree 35 of 1000
building tree 36 of 1000
building tree 37 of 1000
building tree 38 of 1000
building tree 39 of 1000
building tree 40 of 1000
building tree 41 of 1000
building tree 42 of 1000
building tree 43 of 1000
building tree 44 of 1000
building tree 45 of 1000
building tree 46 of 1000
building tree 47 of 1000
building tree 48 of 1000
building tree 49 of 1000
building tree 50 of 1000
building tree 51 of 1000
building tree 52 of 1000
building tree 53 of 1000
building tree 54 of 1000
building tree 55 of 1000
building tree 56 of 1000
building tree 57 of 1000
building tree 58 of 1000
building tree 59 of 1000
building tree 60 of 1000
building tree 61 of 1000
building tree 62 of 1000
building tree 63 of 1000
building tree 64 of 1000
building tree 65 of 1000
building tree 66 of 1000
building tree 67 of 1000
building tree 68 of 1000


[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:   34.3s


...

building tree 509 of 1000
building tree 510 of 1000
building tree 511 of 1000
building tree 512 of 1000
building tree 513 of 1000
building tree 514 of 1000
building tree 515 of 1000
building tree 516 of 1000
building tree 517 of 1000
building tree 518 of 1000
building tree 519 of 1000
building tree 520 of 1000
building tree 521 of 1000
building tree 522 of 1000
building tree 523 of 1000
building tree 524 of 1000
building tree 525 of 1000
building tree 526 of 1000
building tree 527 of 1000
building tree 528 of 1000
building tree 529 of 1000
building tree 530 of 1000
building tree 531 of 1000
building tree 532 of 1000
building tree 533 of 1000
building tree 534 of 1000
building tree 535 of 1000
building tree 536 of 1000
building tree 537 of 1000
building tree 538 of 1000
building tree 539 of 1000
building tree 540 of 1000
building tree 541 of 1000
building tree 542 of 1000
building tree 543 of 1000
building tree 544 of 1000
building tree 545 of 1000
building tree 546 of 1000
building tre

[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  3.7min


building tree 797 of 1000
building tree 798 of 1000
building tree 799 of 1000
building tree 800 of 1000
building tree 801 of 1000
building tree 802 of 1000
building tree 803 of 1000
building tree 804 of 1000
building tree 805 of 1000
building tree 806 of 1000
building tree 807 of 1000
building tree 808 of 1000
building tree 809 of 1000
building tree 810 of 1000
building tree 811 of 1000
building tree 812 of 1000
building tree 813 of 1000
building tree 814 of 1000
building tree 815 of 1000
building tree 816 of 1000
building tree 817 of 1000
building tree 818 of 1000
building tree 819 of 1000
building tree 820 of 1000
building tree 821 of 1000
building tree 822 of 1000
building tree 823 of 1000
building tree 824 of 1000
building tree 825 of 1000
building tree 826 of 1000
building tree 827 of 1000
building tree 828 of 1000
building tree 829 of 1000
building tree 830 of 1000
building tree 831 of 1000
building tree 832 of 1000
building tree 833 of 1000
building tree 834 of 1000
building tre

[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:  4.7min finished


RandomForestRegressor(n_estimators=1000, n_jobs=-1, verbose=3)

In [20]:
submission=pd.DataFrame(index=test.index)
#submission['id']=test.index
num_sold=[round(x) for x in rf_reg.predict(test)]
submission['num_sold']=num_sold
submission.to_csv("submission.csv",index='id')

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed:    0.4s
[Parallel(n_jobs=4)]: Done 280 tasks      | elapsed:    0.8s
[Parallel(n_jobs=4)]: Done 504 tasks      | elapsed:    1.4s
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:    2.2s
[Parallel(n_jobs=4)]: Done 1000 out of 1000 | elapsed:    2.8s finished


In [21]:
submission

,num_sold
id,
136950,36
136951,35
136952,5
136953,36
136954,27
...,...
164320,673
164321,686
164322,104


In [22]:
#ensemble or blenders can be a good idea?